# Used Car Price Prediction using kNN and DNN 

In [3]:
#global imports
import pandas as pd

## Dataset Pre-Processing

### 100,000 UK Used Car Data set

#### Importing the data

In [13]:
audi = pd.read_csv('datasets/UK_Used_Car_Set/audi.csv')
bmw = pd.read_csv('datasets/UK_Used_Car_Set/bmw.csv')
ford = pd.read_csv('datasets/UK_Used_Car_Set/ford.csv')
hyundi = pd.read_csv('datasets/UK_Used_Car_Set/hyundi.csv')
merc = pd.read_csv('datasets/UK_Used_Car_Set/merc.csv')
skoda = pd.read_csv('datasets/UK_Used_Car_Set/skoda.csv')
toyota = pd.read_csv('datasets/UK_Used_Car_Set/toyota.csv')
vauxhall = pd.read_csv('datasets/UK_Used_Car_Set/vauxhall.csv')
vw = pd.read_csv('datasets/UK_Used_Car_Set/vw.csv')

Exploring the dataset

In [15]:
#Storing the lengths
audi_length = len(audi)
bmw_length = len(bmw)

ford_length = len(ford)
hyundi_length = len(hyundi)
merc_length = len(merc)
skoda_length = len(skoda)
toyota_length = len(toyota)
vauxhall_length = len(vauxhall)
vw_length = len(vw)

# Print the lengths
print("Length of audi:", audi_length)
print("Length of bmw:", bmw_length)
print("Length of ford:", ford_length)
print("Length of hyundi:", hyundi_length)
print("Length of merc:", merc_length)
print("Length of skoda:", skoda_length)
print("Length of toyota:", toyota_length)
print("Length of vauxhall:", vauxhall_length)
print("Length of vw:", vw_length)

# Calculate and print the total length
total_length = sum([audi_length, bmw_length, ford_length, hyundi_length, merc_length, skoda_length, toyota_length, vauxhall_length, vw_length])
print("\nTotal size of the Dataset:", total_length)

Length of audi: 10668
Length of bmw: 10781
Length of ford: 17965
Length of hyundi: 4860
Length of merc: 13119
Length of skoda: 6267
Length of toyota: 6738
Length of vauxhall: 13632
Length of vw: 15157

Total size of the Dataset: 99187


Known things to remove in pre-processing
* Engine size 0 - supposed to be for electric cars, if enough values are found this could be removed and made into it's own dataset

In [10]:
audi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10668 entries, 0 to 10667
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         10668 non-null  object 
 1   year          10668 non-null  int64  
 2   price         10668 non-null  int64  
 3   transmission  10668 non-null  object 
 4   mileage       10668 non-null  int64  
 5   fuelType      10668 non-null  object 
 6   tax           10668 non-null  int64  
 7   mpg           10668 non-null  float64
 8   engineSize    10668 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 750.2+ KB
